## Final Project Submission

Please fill out:
* Student name: Charlotte Basch
* Student pace: self paced / part time / <strong> full time </strong>
* Scheduled project review date/time: 
* Instructor name: James Irving 
* Blog post URL:


In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.model_selection import train_test_split
pd.set_option("display.max_rows", None, "display.max_columns", None)

C:\Users\charlotte\anaconda3\envs\learn-env\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
#import data
data = pd.read_csv("kc_house_data.csv")
display(data.head())

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,3,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,3,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,3,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,5,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,3,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [3]:
#View the data types of each column and number of rows and columns in the dataframe
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  float64
 9   view           21534 non-null  float64
 10  condition      21597 non-null  int64  
 11  grade          21597 non-null  int64  
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

In [4]:
#Look for any suspicious values and count the number of unique
#values for each column
for col in data.columns:
    val_counts = data[col].value_counts()
    index_list = []
    value_list = []
    print(f"Number of unique {col} values: {data[col].nunique()}")
    for index, value in val_counts.items():
        if value > 1:
            index_list.append(index)
            value_list.append(value)
    df = pd.DataFrame({col: index_list, "count": value_list})
    df = df.sort_values(by=["count"], ascending=False)
    display(df.head())

Number of unique id values: 21420


,id,count
0,795000620,3
1,1825069031,2
112,643300040,2
113,9238500040,2
114,3935900232,2


Number of unique date values: 372


,date,count
0,6/23/2014,142
2,6/25/2014,131
1,6/26/2014,131
3,7/8/2014,127
4,4/27/2015,126


Number of unique price values: 3622


...

,long,count
0,-122.290,115
1,-122.300,111
2,-122.362,104
3,-122.291,100
4,-122.372,99


Number of unique sqft_living15 values: 777


,sqft_living15,count
0,1540,197
1,1440,195
2,1560,192
3,1500,180
4,1460,169


Number of unique sqft_lot15 values: 8682


,sqft_lot15,count
0,5000,427
1,4000,356
2,6000,288
3,7200,210
4,4800,145


In [5]:
#Given that the sqft_basement variable has a ? in its data, replace it with null values
data = data.replace("?", np.nan)
#Since we have zipcode to indicate location, drop latitude and longitude 
data = data.drop(columns=["lat", "long"])

In [6]:
#See how many null values are in each column
data.isnull().sum()

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement     454
yr_built            0
yr_renovated     3842
zipcode             0
sqft_living15       0
sqft_lot15          0
dtype: int64

In [7]:
#Check the percentage of null values for each column
#An accepted cutoff for dropping null rows is if less than 5% is missing
#If a large amount of data is missing, the column can be dropped
def find_percent_null(df):
    null_nums = df.isnull().sum()
    total_rows = df.shape[0]
    for index, value in null_nums.items():
        if value != 0:
            percent_missing = round((value/total_rows)*100, 2)
            print(f"{percent_missing}% of {index} values are missing.")

#It appears we can drop the missing values for view and sqft_basement but cannot drop any of the columns
find_percent_null(data)

11.0% of waterfront values are missing.
0.29% of view values are missing.
2.1% of sqft_basement values are missing.
17.79% of yr_renovated values are missing.


In [8]:
#Drop the rows where view is missing
data = data.dropna(subset=["view", "sqft_basement"])

#Replace the yr_renovated and waterfront with most common values
data["waterfront"] = data["waterfront"].fillna(data["waterfront"].mode())
data["yr_renovated"] = data["yr_renovated"].fillna(data["yr_renovated"].mode())

In [9]:
#change the data type of sqft_basement
data["sqft_basement"] = data["sqft_basement"].astype("int32")

ValueError: invalid literal for int() with base 10: '0.0'

In [10]:
data.isnull().sum()

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2332
view                0
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3742
zipcode             0
sqft_living15       0
sqft_lot15          0
dtype: int64

In [12]:
#Make columns to indicate the month and year each house was sold


['6', '23', '2014']
['6', '25', '2014']
['7', '8', '2014']
['6', '26', '2014']
